<a href="https://colab.research.google.com/github/JeevaByte/Aws-Data-engineer/blob/main/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

# Enable mixed precision training
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Load and preprocess CIFAR-10 dataset
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))  # Resize for ResNet50
    image = tf.cast(image, tf.float32) / 255.0  # Normalize
    return image, tf.one_hot(label, depth=10)

dataset, info = tfds.load("cifar10", as_supervised=True, with_info=True)
train_ds = dataset["train"].map(preprocess).shuffle(50000).batch(64).prefetch(tf.data.AUTOTUNE)
test_ds = dataset["test"].map(preprocess).batch(64).prefetch(tf.data.AUTOTUNE)

# Distributed training strategy
strategy = tf.distribute.MirroredStrategy()
print("Running on", strategy.num_replicas_in_sync, "devices")

with strategy.scope():
    base_model = tf.keras.applications.ResNet50(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
    base_model.trainable = False  # Freeze pretrained layers

    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    outputs = tf.keras.layers.Dense(10, activation='softmax', dtype='float32')(x)
    model = tf.keras.Model(inputs, outputs)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])

# Custom training loop
def train_step(model, images, labels, loss_fn, optimizer):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_fn(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Training Loop
EPOCHS = 5
for epoch in range(EPOCHS):
    total_loss = 0
    for batch, (images, labels) in enumerate(train_ds):
        loss = train_step(model, images, labels, model.loss, model.optimizer)
        total_loss += loss
        if batch % 100 == 0:
            print(f"Epoch {epoch + 1}, Step {batch}, Loss: {loss.numpy():.4f}")
    print(f"Epoch {epoch + 1} Finished - Avg Loss: {total_loss / len(train_ds):.4f}")

# Evaluate model
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc:.4f}")


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/incomplete.QDXMN5_3.0.2/cifar10-train.tfrecord*...:   0%|         …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/incomplete.QDXMN5_3.0.2/cifar10-test.tfrecord*...:   0%|          …

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.
Running on 1 devices
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
